In [1]:
!wget http://files.grouplens.org/datasets/movielens/ml-25m.zip
!unzip ./ml-25m.zip

--2020-01-23 14:46:05--  http://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  73.0MB/s    in 3.7s    

2020-01-23 14:46:09 (68.1 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]

Archive:  ./ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [2]:
import pandas as pd

ratings = pd.read_csv("./ml-25m/ratings.csv")
movies = pd.read_csv("./ml-25m/movies.csv")

movies_ratings = pd.merge(ratings, movies[['movieId', 'title']], on='movieId')
movies_ratings = movies_ratings.drop(columns="timestamp")
del ratings, movies

movies_ratings.head()

,userId,movieId,rating,title
0,1,296,5.0,Pulp Fiction (1994)
1,3,296,5.0,Pulp Fiction (1994)
2,4,296,4.0,Pulp Fiction (1994)
3,5,296,4.0,Pulp Fiction (1994)
4,7,296,4.0,Pulp Fiction (1994)


In [3]:
ratings_mean_count = pd.DataFrame(movies_ratings.groupby('title')['rating'].mean())
ratings_mean_count['rating_counts'] = pd.DataFrame(movies_ratings.groupby('title')['rating'].count())
ratings_mean_count.sort_values('rating_counts', ascending=False).head(10)

,rating,rating_counts
title,,
Forrest Gump (1994),4.048011,81491
"Shawshank Redemption, The (1994)",4.413576,81482
Pulp Fiction (1994),4.188912,79672
"Silence of the Lambs, The (1991)",4.151342,74127
"Matrix, The (1999)",4.154099,72674
Star Wars: Episode IV - A New Hope (1977),4.120189,68717
Jurassic Park (1993),3.679175,64144
Schindler's List (1993),4.247579,60411
Braveheart (1995),4.002273,59184


In [4]:
ratings_mean_count = ratings_mean_count.reset_index()
ratings_mean_count[ratings_mean_count['rating_counts']>1000].sort_values('rating', ascending=False)

,title,rating,rating_counts
36804,Planet Earth II (2016),4.483096,1124
36803,Planet Earth (2006),4.464797,1747
41782,"Shawshank Redemption, The (1994)",4.413576,81482
4934,Band of Brothers (2001),4.398599,1356
19301,"Godfather, The (1972)",4.324336,52498
...,...,...,...
43601,Spice World (1997),1.805925,2633
14588,Dumb and Dumberer: When Harry Met Lloyd (2003),1.741048,1871
4568,Baby Geniuses (1999),1.699346,1377
5249,Battlefield Earth (2000),1.578374,4268


In [5]:
userid_counts = pd.merge(movies_ratings, ratings_mean_count[['title','rating_counts']], on='title')
userid_counts

,userId,movieId,rating,title,rating_counts
0,1,296,5.0,Pulp Fiction (1994),79672
1,3,296,5.0,Pulp Fiction (1994),79672
2,4,296,4.0,Pulp Fiction (1994),79672
3,5,296,4.0,Pulp Fiction (1994),79672
4,7,296,4.0,Pulp Fiction (1994),79672
...,...,...,...,...,...
25000090,162358,200192,2.0,Den frusna leoparden (1986),1
25000091,162358,200194,2.0,Tough Luck (2004),1
25000092,162386,139970,3.5,I Don't Speak English (1995),1
25000093,162386,200726,4.0,The Graduates (1995),1


In [6]:
user_ratings = userid_counts.drop(userid_counts[userid_counts.	rating_counts < 10000].index)
user_ratings

,userId,movieId,rating,title,rating_counts
0,1,296,5.0,Pulp Fiction (1994),79672
1,3,296,5.0,Pulp Fiction (1994),79672
2,4,296,4.0,Pulp Fiction (1994),79672
3,5,296,4.0,Pulp Fiction (1994),79672
4,7,296,4.0,Pulp Fiction (1994),79672
...,...,...,...,...,...
20583863,162457,4848,3.5,Mulholland Drive (2001),12451
20583864,162473,4848,4.0,Mulholland Drive (2001),12451
20583865,162508,4848,1.5,Mulholland Drive (2001),12451
20583866,162516,4848,3.5,Mulholland Drive (2001),12451


In [7]:
user_movie_rating = user_ratings.pivot_table(index='userId', columns=['title'], values='rating')
user_movie_rating.head()

title,(500) Days of Summer (2009),10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),"40-Year-Old Virgin, The (2005)",50 First Dates (2004),A.I. Artificial Intelligence (2001),About a Boy (2002),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Adaptation (2002),Addams Family Values (1993),"African Queen, The (1951)",Air Force One (1997),Airplane! (1980),Aladdin (1992),Alien (1979),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Almost Famous (2000),Amadeus (1984),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American History X (1998),American Pie (1999),American Pie 2 (2001),"American President, The (1995)",American Psycho (2000),Animal House (1978),Annie Hall (1977),Antz (1998),Apocalypse Now (1979),Apollo 13 (1995),Arachnophobia (1990),Armageddon (1998),Army of Darkness (1993),...,Twister (1996),Unbreakable (2000),Unforgiven (1992),"Untouchables, The (1987)",Up (2009),"Usual Suspects, The (1995)",V for Vendetta (2006),Vanilla Sky (2001),Vertigo (1958),WALL·E (2008),Wag the Dog (1997),Wallace & Gromit: A Close Shave (1995),Wallace & Gromit: The Wrong Trousers (1993),War of the Worlds (2005),Watchmen (2009),Waterworld (1995),Wayne's World (1992),Wedding Crashers (2005),"Wedding Singer, The (1998)",What Women Want (2000),What's Eating Gilbert Grape (1993),When Harry Met Sally... (1989),While You Were Sleeping (1995),Whiplash (2014),Who Framed Roger Rabbit? (1988),Wild Wild West (1999),William Shakespeare's Romeo + Juliet (1996),Willy Wonka & the Chocolate Factory (1971),Witness (1985),"Wizard of Oz, The (1939)","Wolf of Wall Street, The (2013)","X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974),Zombieland (2009),Zoolander (2001)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,4.0,NaN,NaN,NaN,...,NaN,3.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,5.0,3.0,4.0,3.0,3.0,NaN,3.0,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,3.5,NaN,NaN,NaN,5.0,5.0,NaN,2.5,NaN,4.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,5.0,4.0,3.5,NaN,4.0,NaN,5.0,5.0,3.0,4.5,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,4.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,4.0,3.5
4,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,2.5,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.5,NaN,2.5,NaN,NaN,4.0,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,4.0,NaN,4.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
toy_story_ratings = user_movie_rating['Toy Story (1995)']
movies_like_toy_story = user_movie_rating.corrwith(toy_story_ratings)
corr_heading = pd.DataFrame(movies_like_toy_story, columns=['Correlation'])
corr_heading.sort_values('Correlation', ascending=False)

,Correlation
title,
Toy Story (1995),1.000000
Toy Story 2 (1999),0.748456
Toy Story 3 (2010),0.687708
"Monsters, Inc. (2001)",0.562172
Finding Nemo (2003),0.534243
...,...
Johnny Mnemonic (1995),0.030390
Alien: Resurrection (1997),0.029541
Eyes Wide Shut (1999),0.023024
